# Vergleich von Imputation Methoden

An dieser Stelle sollen verschiede Methoden zum interpolieren von fehlenden Werten betrachtet und verglichen werden.

### Vorbereitung

In [1]:
import pandas as pd
import numpy as np
import math
import time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

t0 = time.time()

In [2]:
results = []

In [3]:
def reset_base():
    base= pd.read_csv('additional_data/base.csv') 
    base.set_index(['Country Name', 'Indicator Name'], inplace=True)
    base = base.sort_index(level=['Country Name', 'Indicator Name'])
    return base
base = reset_base()

def get_cords(frac, rnd_state):
    n = int(base.isna().sum().sum()*frac)
    print(f'Testdaten mit {frac*100}% fehlenden Werten (absolut: {n})')
    #random state to ensure reproducibility
    rnds = np.random.RandomState(rnd_state)

    #coordinates for data entries to be removed randomly
    #5000 entries are selected
    cords = pd.DataFrame([[rnds.randint(0, len(base), size=n*4)[i], 
                  rnds.randint(0, len(base.columns), size=n*4)[i]]
                  for i in range(n*4)])

    #all coordinates pointing to NaN entries are removed and
    #first 1000 remaining entries are selected
    cords['value'] = [base.iloc[cords[0][i], cords[1][i]] for i in cords.index]
    cords = cords.dropna()[:n].reset_index(drop=True)
    
    return cords

def reset_train(cords):
    train = reset_base()
    for i in cords.index:
        train.iloc[cords[0][i], cords[1][i]] = None
    return train

def evaluate(df, t, cords):
    
    
    #scaling original data and imputed data
    #necessary ?????????????????????????????????????
    train = reset_train(cords)
    scaler = StandardScaler().fit(train) #fitting on train?
    norm_base = pd.DataFrame(scaler.transform(base))
    df = pd.DataFrame(scaler.transform(df))

    #getting imputed values for simulated NaNs and true value 
    res =pd.DataFrame({'y_true': [norm_base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                       'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index]
                      })
    res = res.dropna()

   
    #calculate evaluation metrics
    r2 = r2_score(res['y_true'], res['y_pred'])
    rmse = math.sqrt(mean_squared_error(res['y_true'], res['y_pred']))
    still_missing = df.isna().sum().sum()
    

    print(f'r2: {r2}, rmse: {rmse}, t: {t}')
    print('')
    
    return [r2, rmse, still_missing, t]



In [4]:
def impute_backfill(df):
    df = df.fillna(method='bfill', limit=3)
    return df

def impute_overall_means(df):
    #fill NaNs with overall mean of that indicator
    values = pd.DataFrame(df.stack()).groupby('Indicator Name')[0].mean()
    df = pd.DataFrame(df.stack(dropna=False))
    
    df[0] = df[0].fillna(df.groupby('Indicator Name')[0].transform('mean'))
    df = df.unstack()
    df.columns = df.columns.droplevel(0)
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
        
    return df

def impute_yearly_means(df):
    #fill NaNs with overall mean of that indicator
    
    for i in df.columns:
        df[i] = df[i].fillna(df.groupby('Indicator Name')[i].transform('mean'))
            
    return df

def impute_yearly_means_per_region(df):
    country_data = pd.read_csv('../Data/WDICountry.csv')
    country_data = country_data.loc[:,['Table Name', 'Region']]
    df = pd.merge(df.reset_index(), country_data, how='left', left_on='Country Name', right_on='Table Name').drop('Table Name', axis=1)
    df = df.set_index(['Country Name', 'Indicator Name', 'Region'])

    for i in df.columns:
        df[i] = df[i].fillna(df.groupby(['Indicator Name', 'Region'])[i].transform('mean'))

    df = df.reset_index().set_index(['Country Name', 'Indicator Name']).drop('Region', axis=1)
    return df

def interpolate3(df):
    df = df.interpolate(limit=3)
    return df

def interpolate_all(df):
    df = df.interpolate()
    return df

def iterative_imputer_1(df):
    col = df.columns
    idx = df.index
    
    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)
    df= pd.DataFrame(df, columns=col, index=idx)
    return df

def iterative_imputer_2(df):
    df = df.unstack().T
    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

def iterative_imputer_3(df):

    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999, verbose=True)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

def mice_imputer(df):
    n_imputations =  12
    dfs = []
    col = df.columns
    idx = df.index
    
    for i in range(n_imputations): 
        print(f'Imputation round {i}')
        iter_imp = IterativeImputer(random_state=i, sample_posterior=True, verbose=2)
        df_temp = iter_imp.fit_transform(df)
        dfs.append(df_temp)
    
    df = np.mean(np.array(dfs), axis=0)
    df = pd.DataFrame(df, columns=col, index=idx)
    return df

def knn_imputer1(df):
    col = df.columns
    idx = df.index
    
    knn_imp = KNNImputer(n_neighbors=2)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)
    return df

def knn_imputer2(df, n=4):
    
    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    knn_imp = KNNImputer(n_neighbors=n)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)

    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

In [5]:
functions =  {'Backfill':impute_backfill, 'Overall Mean':impute_overall_means, 
              'Yearly Mean':impute_yearly_means, 'Yearly Mean per Region':impute_yearly_means_per_region, 
              'Interpolate 3':interpolate3, 'Interpolate all':interpolate_all, 
              'ICE 1':iterative_imputer_1, 'ICE 2':iterative_imputer_2, 
              'ICE 3':iterative_imputer_3, 'MICE':mice_imputer, 
              'KNN 1':knn_imputer1, 'KNN 2':knn_imputer2
             }

In [6]:
def run_all(frac):
    
    cords_list =[]
    n = 7
    for i in np.arange(1,n+1):
        cords_list.append(get_cords(frac, i))
    
    results = {}
    detailed_results = {}    
    
    for name, func in functions.items():
        func_runs = []
        print(name)
        
        for count, cords in enumerate(cords_list):
            
            print(f'run: {count+1}/{n}')
            t0 = time.time()
            df= func(reset_train(cords)) 
            t1 = time.time()

            t = t1-t0

            func_runs.append(evaluate(df, t, cords))
        
        print(func_runs)
        print ('')
        detailed_results[name] = func_runs
        results[name] = np.mean(np.array(func_runs), axis=0)
        
    return results, detailed_results


In [7]:
r1, r1_detail = run_all(0.05)

Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Backfill
run: 0
r2: -0.2630386067986965, rmse: 3.1264580984032304, t: 0.8018028736114502

run: 1
r2: -2.77533240911633, rmse: 1.1896157293266358, t: 0.7937955856323242

run: 2
r2: -1.688280205196218, rmse: 1.0226797653748734, t: 0.793506383895874

run: 3
r2: -0.6672076075877178, rmse: 2.085863590967128, t: 0.7936129570007324

run: 4
r2: -0.07720522781720884, rmse: 3.533882957257935, t: 0.7985427379608154

[[-0.2630386067986965, 3.1264580984032304, 32924, 0.8018028736114502], [-2.77533240911633, 1.1896157293266358, 32825, 0.7937955856323242], [-1.688280205196218, 1.0226797653748734, 32824, 0.793506383895874], [-0.6672076075877178, 2.085863590967128, 32841, 0.7936129570007324], [-0.07720522781720884, 3.533882957257935, 32828, 0.7985

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9942158016960598, rmse: 0.20756410705991854, t: 9.451332092285156

run: 1


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.7827329807258437, rmse: 0.2820413273792655, t: 9.43781852722168

run: 2


...

[IterativeImputer] Ending imputation round 4/10, elapsed time 4.38
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.50
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.57
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.67
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.72
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.81
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.89
Imputation round 11
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.12
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.21
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.27
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.33
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.41
[IterativeImputer] Ending imputation round 7/10

[IterativeImputer] Ending imputation round 2/10, elapsed time 2.14
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.26
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.34
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.45
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.53
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.60
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.69
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.78
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.85
Imputation round 10
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.08
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.16
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.28
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.38
[IterativeImputer] Ending imputation round 5/10

[IterativeImputer] Ending imputation round 1/10, elapsed time 1.09
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.19
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.25
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.33
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.39
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.48
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.55
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.63
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.72
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.81
Imputation round 9
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.08
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.19
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.24
[IterativeImputer] Ending imputation round 4/10,

[IterativeImputer] Ending imputation round 10/10, elapsed time 10.90
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.16
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.26
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.35
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.41
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.49
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.59
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.68
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.78
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.94
Imputation round 8
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.08
[IterativeImputer] Ending imputati

[IterativeImputer] Ending imputation round 8/10, elapsed time 8.80
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.92
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.00
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.10
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.19
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.32
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.42
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.52
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.59
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.70
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.78
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.85
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.97
Imputation round 7
[IterativeImputer] Completi

In [8]:
r1 = pd.DataFrame(r1).T
r1

,0,1,2,3
Backfill,-1.094213,2.191700,32848.4,0.796252
Overall Mean,-0.251541,1.805936,0.0,1.041184
Yearly Mean,0.009964,1.768969,52298.0,0.879176
Yearly Mean per Region,0.016326,1.770021,58352.6,0.992067
Interpolate 3,-0.557857,1.999203,32879.0,0.861664
Interpolate all,-0.557853,1.966919,58.0,0.840356
ICE 1,0.948744,0.210681,0.0,9.420938
ICE 2,0.926469,0.448741,0.0,22.879489
ICE3,0.703721,0.927913,0.0,82.013502
MICE,0.951523,0.236147,0.0,132.102118


In [9]:
r1_detail = pd.DataFrame(pd.DataFrame(r1_detail).T.stack().to_dict()).T
r1_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r1_detail

r2      rmse  still_missing           t
Backfill               0 -0.263039  3.126458        32924.0    0.801803
                       1 -2.775332  1.189616        32825.0    0.793796
                       2 -1.688280  1.022680        32824.0    0.793506
                       3 -0.667208  2.085864        32841.0    0.793613
                       4 -0.077205  3.533883        32828.0    0.798543
Overall Mean           0  0.056692  2.650681            0.0    1.042257
                       1 -0.790210  0.809595            0.0    1.045049
                       2 -0.553206  0.765509            0.0    1.039034
                       3 -0.089972  1.660006            0.0    1.036537
                       4  0.118993  3.143888            0.0    1.043042
Yearly Mean            0  0.008985  2.716882        52298.0    0.873251
                       1 -0.003552  0.606158        52298.0    0.889392
                       2  0.025439  0.606373        52298.0    0.874874
                       3  0.008787  1.583016        52298.0    0.880491
                       4  0.010162  3.332418        52298.0    0.877872
Yearly Mean per Region 0  0.043249  2.697510        58365.0    0.985987
                       1 -0.133955  0.651156        58318.0    1.002857
                       2  0.142235  0.575184        58345.0    0.985069
                       3 -0.016975  1.620865        58359.0    0.998668
                       4  0.047073  3.305390        58376.0    0.987751
Interpolate 3          0 -0.072788  2.873685        32955.0    0.865369
                       1 -1.525109  0.977054        32855.0    0.860858
                       2 -0.930706  0.867540        32855.0    0.861359
                       3 -0.198324  1.769570        32872.0    0.866372
                       4 -0.062355  3.508164        32858.0    0.854361
Interpolate all        0 -0.072774  2.826733           58.0    0.839828
                       1 -1.525022  0.961498           58.0    0.844934
                       2 -0.930811  0.853504           58.0    0.839834
                       3 -0.198316  1.740554           58.0    0.838836
                       4 -0.062341  3.452306           58.0    0.838348
ICE 1                  0  0.994216  0.207564            0.0    9.451332
                       1  0.782733  0.282041            0.0    9.437819
                       2  0.988237  0.066620            0.0    9.369901
                       3  0.987720  0.176199            0.0    9.425428
                       4  0.990817  0.320980            0.0    9.420213
ICE 2                  0  0.897000  0.875889            0.0   22.742195
                       1  0.966479  0.110783            0.0   22.920295
                       2  0.850137  0.237784            0.0   22.966735
                       3  0.965368  0.295899            0.0   23.002361
                       4  0.953361  0.723353            0.0   22.765858
ICE3                   0  0.672465  1.561922            0.0   80.134166
                       1  0.864944  0.222368            0.0   82.428435
                       2  0.436893  0.460926            0.0   82.521717
                       3  0.793355  0.722794            0.0   82.359860
                       4  0.750950  1.671555            0.0   82.623334
MICE                   0  0.992594  0.234863            0.0  132.384988
                       1  0.815449  0.259941            0.0  131.406286
                       2  0.993037  0.051253            0.0  132.091313
                       3  0.967271  0.287652            0.0  132.045735
                       4  0.989266  0.347026            0.0  132.582266
KNN 1                  0  0.549804  1.831182            0.0  103.620099
                       1  0.155114  0.556180            0.0  100.831530
                       2  0.358112  0.492113            0.0  100.612728
                       3  0.018670  1.575105            0.0  101.903874
                       4  0.282422  2.837343            0.0  101.914644


In [10]:
r2 , r2_detail =  run_all(0.075)

Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Backfill
run: 0
r2: -0.7622360204251646, rmse: 1.9101575370444133, t: 1.104684829711914

run: 1
r2: -0.13326755926364164, rmse: 1.5732663625326984, t: 1.1138923168182373

run: 2
r2: -0.15471382925960686, rmse: 2.34729184379117, t: 1.1242446899414062

run: 3
r2: -0.38762500614250417, rmse: 2.1233606768359277, t: 1.1117191314697266

run: 4
r2: -0.12638032635757868, rmse: 1.8674634798834449, t: 1.1321732997894287

[[-0.7622360204251646, 1.9101575370444133, 33252, 1.104684829711914], [-0.13326755926364164, 1.5732663625326984, 33280, 1.1138923168182373], [-0.15471382925960686, 2.34729184379117, 33292, 1.1242446899414062], [-0.38762500614250417, 2.1233606768359277, 33312, 1.1117191314697266], [-0.12638032635757868, 1.86746347988344

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9803957096839419, rmse: 0.19838139998923512, t: 9.555473566055298

run: 1


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9755494244776735, rmse: 0.22712199683626336, t: 9.513757944107056

run: 2


...

[IterativeImputer] Ending imputation round 5/10, elapsed time 5.36
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.46
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.52
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.61
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.69
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.77
Imputation round 11
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.13
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.20
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.27
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.31
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.39
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.48
[IterativeImputer] Ending imputation round 8/10

[IterativeImputer] Ending imputation round 3/10, elapsed time 3.25
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.33
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.40
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.46
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.55
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.62
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.72
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.80
Imputation round 10
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.09
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.14
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.24
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.31
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.40
[IterativeImputer] Ending imputation round 6/10

[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.16
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.21
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.30
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.36
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.45
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.54
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.64
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.71
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.81
Imputation round 9
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.06
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.14
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.20
[IterativeImputer] Ending imputation round 4/10,

[IterativeImputer] Ending imputation round 10/10, elapsed time 10.65
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.05
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.11
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.18
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.24
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.34
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.40
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.49
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.57
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.64
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.70
Imputation round 8
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.08
[IterativeImputer] Ending imputati

[IterativeImputer] Ending imputation round 8/10, elapsed time 8.51
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.58
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.66
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.15
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.22
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.32
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.40
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.48
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.56
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.65
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.71
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.80
Imputation round 7
[IterativeImputer] Completi

In [11]:
r2 = pd.DataFrame(r2).T
r2

,0,1,2,3
Backfill,-0.312845,1.964308,33265.4,1.117343
Overall Mean,-0.052291,1.740381,0.0,1.360476
Yearly Mean,0.014764,1.692044,52298.0,1.200444
Yearly Mean per Region,0.049003,1.681218,58425.0,1.309644
Interpolate 3,-0.219687,1.909891,33296.8,1.203304
Interpolate all,-0.221227,1.880009,59.6,1.168807
ICE 1,0.986040,0.182850,0.0,9.550186
ICE 2,0.945163,0.365983,0.0,23.290719
ICE3,0.785148,0.771223,0.0,81.028156
MICE,0.988788,0.168885,0.0,130.558931


In [12]:
r2_detail = pd.DataFrame(pd.DataFrame(r2_detail).T.stack().to_dict()).T
r2_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r2_detail

r2      rmse  still_missing           t
Backfill               0 -0.762236  1.910158        33252.0    1.104685
                       1 -0.133268  1.573266        33280.0    1.113892
                       2 -0.154714  2.347292        33292.0    1.124245
                       3 -0.387625  2.123361        33312.0    1.111719
                       4 -0.126380  1.867463        33191.0    1.132173
Overall Mean           0 -0.122387  1.501055            0.0    1.353251
                       1 -0.088914  1.515694            0.0    1.363470
                       2  0.032755  2.111409            0.0    1.354509
                       3 -0.007757  1.778103            0.0    1.378921
                       4 -0.075152  1.795642            0.0    1.352227
Yearly Mean            0  0.018221  1.403888        52298.0    1.205419
                       1  0.024126  1.434867        52298.0    1.196822
                       2  0.003268  2.143351        52298.0    1.196841
                       3  0.018274  1.754989        52298.0    1.204749
                       4  0.009931  1.723128        52298.0    1.198388
Yearly Mean per Region 0  0.093030  1.363607        58419.0    1.304992
                       1  0.055061  1.426347        58433.0    1.310894
                       2  0.008738  2.157995        58406.0    1.302746
                       3  0.064649  1.729697        58428.0    1.308520
                       4  0.023535  1.728443        58439.0    1.321066
Interpolate 3          0 -0.251263  1.612834        33283.0    1.180279
                       1 -0.246770  1.647952        33312.0    1.194232
                       2 -0.177726  2.369175        33323.0    1.208258
                       3 -0.247715  2.011962        33344.0    1.252429
                       4 -0.174961  1.907532        33222.0    1.181322
Interpolate all        0 -0.251251  1.584884           58.0    1.195201
                       1 -0.246734  1.621943           65.0    1.157874
                       2 -0.185513  2.337530           58.0    1.162788
                       3 -0.247688  1.978560           59.0    1.162372
                       4 -0.174948  1.877128           58.0    1.165801
ICE 1                  0  0.980396  0.198381            0.0    9.555474
                       1  0.975549  0.227122            0.0    9.513758
                       2  0.993720  0.170131            0.0    9.543736
                       3  0.983162  0.229840            0.0    9.572116
                       4  0.997372  0.088775            0.0    9.565849
ICE 2                  0  0.983851  0.180055            0.0   23.370723
                       1  0.987873  0.159954            0.0   23.471527
                       2  0.822200  0.905253            0.0   23.305643
                       3  0.939175  0.436837            0.0   23.069102
                       4  0.992714  0.147817            0.0   23.236597
ICE3                   0  0.862321  0.525726            0.0   82.119667
                       1  0.924897  0.398056            0.0   82.020792
                       2  0.411709  1.646646            0.0   81.239880
                       3  0.892430  0.580931            0.0   80.149082
                       4  0.834381  0.704758            0.0   79.611359
MICE                   0  0.987011  0.161477            0.0  130.232394
                       1  0.981214  0.199080            0.0  130.690228
                       2  0.994802  0.154784            0.0  130.738956
                       3  0.985241  0.215186            0.0  130.726423
                       4  0.995674  0.113899            0.0  130.406657
KNN 1                  0  0.052213  1.379370            0.0  101.757998
                       1  0.133435  1.352120            0.0  101.379353
                       2  0.513783  1.496991            0.0  101.323126
                       3  0.270549  1.512785            0.0  101.135177
                       4  0.039331  1.697351            0.0  101.156196


In [13]:
r3, r3_detail = run_all(0.1)

Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Backfill
run: 0
r2: -3.964219549140708, rmse: 1.1521878884113992, t: 1.4466934204101562

run: 1
r2: -1.8373707535864638, rmse: 1.6792466658151293, t: 1.436490535736084

run: 2
r2: -1.9081389310564236, rmse: 1.5239926108614468, t: 1.4265525341033936

run: 3
r2: -0.8977876217879388, rmse: 1.5383638773729555, t: 1.4446756839752197

run: 4
r2: -1.1760918948068961, rmse: 1.4134849682042556, t: 1.466137409210205

[[-3.964219549140708, 1.1521878884113992, 33619, 1.4466934204101562], [-1.8373707535864638, 1.6792466658151293, 33786, 1.436490535736084], [-1.9081389310564236, 1.5239926108614468, 33654, 1.4265525341033936], [-0.8977876217879388, 1.5383638773729555, 33643, 1.4446756839752197], [-1.1760918948068961, 1.4134849682042556

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9370202334659707, rmse: 0.12777336992410396, t: 9.834977388381958

run: 1


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


r2: 0.9521427024799756, rmse: 0.2140660895293748, t: 9.787731409072876

run: 2


...

[IterativeImputer] Ending imputation round 4/10, elapsed time 4.32
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.39
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.48
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.53
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.62
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.69
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.79
Imputation round 11
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.16
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.23
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.30
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.35
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.43
[IterativeImputer] Ending imputation round 7/10

[IterativeImputer] Ending imputation round 2/10, elapsed time 2.15
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.23
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.31
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.39
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.47
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.57
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.64
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.72
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.79
Imputation round 10
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.09
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.16
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.25
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.34
[IterativeImputer] Ending imputation round 5/10

[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.15
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.23
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.30
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.41
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.49
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.59
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.64
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.73
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.79
Imputation round 9
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.08
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.15
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.25
[IterativeImputer] Ending imputation round 4/10,

[IterativeImputer] Ending imputation round 10/10, elapsed time 10.79
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.08
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.18
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.26
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.34
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.40
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.48
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.54
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.63
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.69
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.77
Imputation round 8
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputati

[IterativeImputer] Ending imputation round 8/10, elapsed time 8.66
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.73
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.83
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.07
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.16
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.23
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.32
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.37
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.47
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.51
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.60
[IterativeImputer] Ending imputation round 9/10, elapsed time 9.69
[IterativeImputer] Ending imputation round 10/10, elapsed time 10.78
Imputation round 7
[IterativeImputer] Completi

In [14]:
r3 = pd.DataFrame(r3).T
r3

,0,1,2,3
Backfill,-1.956722,1.461455,33676.2,1.444110
Overall Mean,-0.422216,1.018547,0.0,1.686650
Yearly Mean,0.003620,0.877554,52329.6,1.516319
Yearly Mean per Region,-0.067946,0.905732,58630.2,1.620215
Interpolate 3,-0.891110,1.178038,33706.8,1.502049
Interpolate all,-0.892725,1.159193,58.4,1.485679
ICE 1,0.956064,0.166354,0.0,9.837879
ICE 2,0.958219,0.167993,0.0,23.388888
ICE3,0.736991,0.452677,0.0,81.503792
MICE,0.967982,0.144287,0.0,131.616426


In [15]:
r3_detail = pd.DataFrame(pd.DataFrame(r3_detail).T.stack().to_dict()).T
r3_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r3_detail

r2      rmse  still_missing           t
Backfill               0 -3.964220  1.152188        33619.0    1.446693
                       1 -1.837371  1.679247        33786.0    1.436491
                       2 -1.908139  1.523993        33654.0    1.426553
                       3 -0.897788  1.538364        33643.0    1.444676
                       4 -1.176092  1.413485        33679.0    1.466137
Overall Mean           0 -1.083659  0.734942            0.0    1.685571
                       1 -0.203727  1.073588            0.0    1.687289
                       2 -0.364231  1.025791            0.0    1.689113
                       3 -0.178540  1.191847            0.0    1.684852
                       4 -0.280922  1.066565            0.0    1.686424
Yearly Mean            0 -0.037370  0.518569        52298.0    1.507259
                       1  0.015803  0.970766        52298.0    1.527660
                       2  0.003029  0.876938        52456.0    1.513745
                       3  0.018013  1.087931        52298.0    1.518711
                       4  0.018624  0.933563        52298.0    1.514221
Yearly Mean per Region 0 -0.373088  0.603081        58608.0    1.612767
                       1  0.051644  0.963471        58578.0    1.618765
                       2  0.015290  0.881940        58645.0    1.625893
                       3 -0.026592  1.123984        58586.0    1.617838
                       4 -0.006983  0.956185        58734.0    1.625813
Interpolate 3          0 -2.091582  0.910245        33649.0    1.514238
                       1 -0.559111  1.242379        33816.0    1.488905
                       2 -0.762802  1.185136        33686.0    1.515258
                       3 -0.426421  1.333826        33674.0    1.485639
                       4 -0.615634  1.218606        33709.0    1.506205
Interpolate all        0 -2.091558  0.895217           58.0    1.501225
                       1 -0.559080  1.221822           58.0    1.485716
                       2 -0.762587  1.166012           59.0    1.484190
                       3 -0.434792  1.315052           58.0    1.477114
                       4 -0.615608  1.197862           59.0    1.480149
ICE 1                  0  0.937020  0.127773            0.0    9.834977
                       1  0.952143  0.214066            0.0    9.787731
                       2  0.989395  0.090442            0.0    9.854593
                       3  0.987255  0.123943            0.0    9.847285
                       4  0.914507  0.275545            0.0    9.864806
ICE 2                  0  0.935392  0.129414            0.0   23.000120
                       1  0.975435  0.153368            0.0   23.263726
                       2  0.936051  0.222092            0.0   23.077013
                       3  0.978803  0.159841            0.0   23.270274
                       4  0.965416  0.175252            0.0   24.333307
ICE3                   0  0.865651  0.186619            0.0   82.098812
                       1  0.787376  0.451211            0.0   81.228597
                       2  0.680454  0.496457            0.0   81.756182
                       3  0.479221  0.792274            0.0   81.329163
                       4  0.872251  0.336826            0.0   81.106206
MICE                   0  0.961929  0.099342            0.0  131.199714
                       1  0.955398  0.206657            0.0  132.180218
                       2  0.989838  0.088533            0.0  131.461364
                       3  0.992163  0.097189            0.0  131.497082
                       4  0.940582  0.229712            0.0  131.743752
KNN 1                  0  0.135463  0.473404            0.0  103.851465
                       1  0.161437  0.896069            0.0  103.529304
                       2  0.515733  0.611164            0.0  103.526942
                       3  0.145595  1.014800            0.0  103.580086
                       4  0.202876  0.841374            0.0  103.279538


In [16]:
t1 = time.time()
t = t1-t0
print(f'comlete runtime: {t/60}min')

comlete runtime: 108.46655972798665min


In [17]:
sets = {'r1':r1, 'r1_detail':r1_detail,
       'r2':r2, 'r2_detail':r2_detail,
       'r3':r3, 'r3_detail':r3_detail,}
for name, df in sets.items():
    df.to_csv(f'additional_data/results/{name}.csv')